In [293]:
# Paso 1.
# Realice la limpieza de los datasets:
import pandas as pd
import numpy as np

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)
# - Renombre la columna "default payment next month" a "default".

dataset_test.rename(columns={"default payment next month": "default"}, inplace=True)
dataset_train.rename(columns={"default payment next month": "default"}, inplace=True)


# - Remueva la columna "ID".

dataset_test.pop("ID")
dataset_train.pop("ID")

dataset_train['EDUCATION'] = dataset_train['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)
dataset_test['EDUCATION'] = dataset_test['EDUCATION'].apply(lambda x: x if x > 0 else np.nan)

dataset_train['MARRIAGE'] = dataset_train['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)
dataset_test['MARRIAGE'] = dataset_test['MARRIAGE'].apply(lambda x: x if x > 0 else np.nan)


# - Elimine los registros con informacion no disponible.

dataset_test = dataset_test.dropna()
dataset = dataset_train.dropna()


# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".

dataset_test['EDUCATION'] = dataset_test['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
dataset_train['EDUCATION'] = dataset_train['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

In [294]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
#
y_train=dataset_train.copy()
y_train=y_train.pop("default")

x_train = dataset_train.copy()
x_train.pop("default")

y_test=dataset_test.copy()
y_test=y_test.pop("default")

x_test = dataset_test.copy()
x_test.pop("default")

0       1
1       0
2       0
3       0
4       0
       ..
8995    0
8996    0
8997    0
8998    1
8999    1
Name: default, Length: 8979, dtype: int64

In [295]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Descompone la matriz de entrada usando componentes principales.
#   El pca usa todas las componentes.
# - Escala la matriz de entrada al intervalo [0, 1].
# - Selecciona las K columnas mas relevantes de la matrix de entrada.
# - Ajusta una red neuronal tipo MLP.

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

# Definir las columnas categóricas y numéicas
categorical_columns = ["SEX", "EDUCATION", "MARRIAGE"]
numeric_columns = [col for col in x_train.columns if col not in categorical_columns]


    
# Configurar transformación por columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns),  # One-hot encoding para categóricas
        ('num', StandardScaler(), numeric_columns)   
    ],
    remainder='passthrough'  # Mantener las demás columnas numéricas sin procesar
)

#Selección de las K mejores características
k_best_selector = SelectKBest(score_func=f_classif)

# Crear el pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),   
    ('pca', PCA()),                              
    ('feature_selector', k_best_selector), 
    ('mlp', MLPClassifier(max_iter=200, random_state=42)),                      
])


# Entrenar el pipeline con los datos de entrenamiento
pipeline.fit(x_train, y_train)


print("\nPipeline de clasificación ajustado exitosamente.")



Pipeline de clasificación ajustado exitosamente.


c:\Users\usuario\Downloads\2024-2-LAB-04-prediccion-del-default-usando-mlp-dzaptaca\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [296]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'pca__n_components':[18],
    'feature_selector__k':[18],
    'mlp__hidden_layer_sizes': [(140,20)],
    'mlp__activation': ['logistic'],  ## 'identity', 'logistic', 'tanh', 'relu'
    'mlp__solver': ['adam'],   ## 'lbfgs', 'sgd', 'adam'
    'mlp__alpha': [0.0006],  
    'mlp__learning_rate': ['constant'],  ## 'constant', 'invscaling', 'adaptive'
}

# Crear un estimador GridSearchCV con validación cruzada
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='balanced_accuracy',  
    cv=10,                                        
    n_jobs=-1,                                  
    verbose=1                                
)

# Ajustar el GridSearchCV con los datos de entrenamiento
grid_search.fit(x_train, y_train)

# Mostrar los mejores parámetros encontrados y la puntuación
print("\nMejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"Mejor precisión balanceada (validación cruzada): {grid_search.best_score_:.4f}")

# Actualizar el pipeline con los mejores parámetros encontrados
best_pipeline = grid_search.best_estimator_



Fitting 10 folds for each of 1 candidates, totalling 10 fits

Mejores parámetros encontrados:
{'feature_selector__k': 18, 'mlp__activation': 'logistic', 'mlp__alpha': 0.0006, 'mlp__hidden_layer_sizes': (140, 20), 'mlp__learning_rate': 'constant', 'mlp__solver': 'adam', 'pca__n_components': 18}
Mejor precisión balanceada (validación cruzada): 0.6587


c:\Users\usuario\Downloads\2024-2-LAB-04-prediccion-del-default-usando-mlp-dzaptaca\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [297]:
import os
import pickle
import gzip

dir_path = '../files/models'

#os.makedirs(os.path.dirname(dir_path), exist_ok=True) # Otra forma de crear/verificar la direccion

if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(grid_search, f)
else:
    with gzip.open('../files/models/model.pkl.gz', 'wb') as f:
        pickle.dump(grid_search, f)


In [298]:
import os
import json
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score

# Paso 6: Calcular métricas y guardar en un archivo JSON, asegurándose de que la ruta exista

# Definir la función para calcular métricas
def calculate_metrics(y_true, y_pred, dataset_name):
    metrics = {
        'type': 'metrics',
        'dataset': dataset_name,
        'precision': precision_score(y_true, y_pred, average='binary'),
        'balanced_accuracy': balanced_accuracy_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred, average='binary'),
        'f1_score': f1_score(y_true, y_pred, average='binary'),
    }
    return metrics

# Generar predicciones para los conjuntos de entrenamiento y prueba
y_train_pred = best_pipeline.predict(x_train)
y_test_pred = best_pipeline.predict(x_test)

# Calcular métricas para cada conjunto
train_metrics = calculate_metrics(y_train, y_train_pred, 'train')
test_metrics = calculate_metrics(y_test, y_test_pred, 'test')

# Definir la ruta del archivo de salida
output_dir = "../files/output"
output_path = os.path.join(output_dir, "metrics.json")

# Crear las carpetas necesarias si no existen
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
with open(output_path, 'w') as f:
    json.dump(train_metrics, f)
    f.write("\n")
    json.dump(test_metrics, f)


print(f"\nMétricas guardadas exitosamente en '{output_path}'")



Métricas guardadas exitosamente en '../files/output\metrics.json'


In [299]:
import os
import json
from sklearn.metrics import confusion_matrix

# Paso 7: Calcular matrices de confusión y añadirlas al archivo JSON

# Función para generar la matriz de confusión en el formato requerido
def format_confusion_matrix(cm, dataset_name):
    return {
        'type': 'cm_matrix',
        'dataset': dataset_name,
        'true_0': {
            'predicted_0': int(cm[0, 0]),
            'predicted_1': int(cm[0, 1])
        },
        'true_1': {
            'predicted_0': int(cm[1, 0]),
            'predicted_1': int(cm[1, 1])
        }
    }

# Calcular las matrices de confusión
cm_train = confusion_matrix(y_train, y_train_pred)
cm_test = confusion_matrix(y_test, y_test_pred)

# Formatear las matrices de confusión
train_cm_metrics = format_confusion_matrix(cm_train, 'train')
test_cm_metrics = format_confusion_matrix(cm_test, 'test')

# Definir la ruta del archivo para las matrices de confusión
confusion_matrix_dir = "../files/output"
confusion_matrix_path = os.path.join(confusion_matrix_dir, "metrics.json")

# Crear las carpetas necesarias si no existen
os.makedirs(confusion_matrix_dir, exist_ok=True)

# Cargar métricas existentes o inicializar una lista vacía si el archivo está vacío o no existe
metrics = []
if os.path.exists(confusion_matrix_path):
    try:
        with open(confusion_matrix_path, 'r') as f:
            content = f.read().strip()  # Leer contenido del archivo
            if content:  # Validar que no esté vacío
                metrics = json.loads(content)
    except (json.JSONDecodeError, ValueError):
        print(f"Advertencia: El archivo '{confusion_matrix_path}' tiene un formato inválido. Será sobrescrito.")

# Añadir las matrices de confusión al archivo


# Guardar el archivo actualizado
with open(confusion_matrix_path, 'a') as f:
    f.write("\n")
    json.dump(train_cm_metrics, f)
    f.write("\n")
    json.dump(test_cm_metrics, f)

print(f"\nMatrices de confusión guardadas exitosamente en '{confusion_matrix_path}'")


Advertencia: El archivo '../files/output\metrics.json' tiene un formato inválido. Será sobrescrito.

Matrices de confusión guardadas exitosamente en '../files/output\metrics.json'
